In [1]:
from ase.build import fcc111, add_adsorbate, bulk
from ase.calculators.emt import EMT
from ase.optimize import MDMin, BFGS
from ase.constraints import FixAtoms
from ase.db import connect
from ase.visualize import view
from ase.neb import NEB
import numpy as np
from ase.io.trajectory import Trajectory
from matplotlib import pyplot as plt
%matplotlib inline

### Create intial and final configs

In [2]:
Pt_slab = fcc111('Pt', size=(8, 7, 6), a=3.9)

init_sub_ids = [[315, 316, 323], [316, 317, 324], [307, 308, 315], 
                [308, 309, 316], [309, 310, 317], [300, 301, 308], 
                [301, 302, 309]]

Pt_slab_init = Pt_slab.copy()
for ids in init_sub_ids:
    xy_init = Pt_slab.positions[ids][:, :2].mean(axis=0)
    add_adsorbate(Pt_slab_init, 'Pt', position=xy_init, height=1.5)
const = FixAtoms([atom.index for atom in Pt_slab_init if atom.index < 168])
Pt_slab_init.set_constraint(const)
Pt_slab_init.set_calculator(EMT())
Pt_slab_init.center(vacuum=10.0, axis=2)


final_sub_ids = [[314, 315, 322], [315, 316, 323], [307, 308, 315], 
                 [308, 309, 316], [316, 317, 324], [300, 301, 308],  
                 [301, 302, 309]]

Pt_slab_final = Pt_slab.copy()
for ids in final_sub_ids:
    xy_init = Pt_slab.positions[ids][:, :2].mean(axis=0)
    add_adsorbate(Pt_slab_final, 'Pt', position=xy_init, height=1.5)
const = FixAtoms([atom.index for atom in Pt_slab_final if atom.index < 168])
Pt_slab_final.set_constraint(const)
Pt_slab_final.set_calculator(EMT())
Pt_slab_final.center(vacuum=10.0, axis=2)

view(Pt_slab_init)
view(Pt_slab_final)

In [3]:
dyn = BFGS(Pt_slab_init, logfile='./datasets/BFGS-Pt-log-init.txt')
dyn.run(fmax=0.05)

dyn = BFGS(Pt_slab_final, logfile='./datasets/BFGS-Pt-log-final.txt')
dyn.run(fmax=0.05)

True

In [4]:
n_inter_images = 9

# Make a band consisting of 11(2+9) images:
images = [Pt_slab_init]
images += [Pt_slab_init.copy() for i in range(n_inter_images)]
images += [Pt_slab_final]

for image in images[1:1+n_inter_images]:
    image.set_calculator(EMT())

neb = NEB(images, k=1, climb=True)
neb.interpolate()

# save the initial images to be used in NN ensemble method
init_db = connect('./datasets/Pt-init-images.db')
for image in images:
    init_db.write(image)

# Optimize:
optimizer = MDMin(neb, trajectory='./datasets/Pt-MDMin.traj', logfile='./datasets/Pt-MDMin.txt')
optimizer.run(fmax=0.05)

True

In [ ]:
traj = Trajectory('./datasets/Pt-MDmin-EMT.traj', 'r')
traj_nrgs = []
traj_forces = []

for entry in traj:
    traj_nrgs.append(entry.get_potential_energy())
    traj_forces.append(entry.get_forces())

In [ ]:
all_nrgs = []
all_forces = []
all_images = []
for i in range(66):
    temp_nrgs = []
    temp_forces = []
    temp_images = []
    for entry in list(traj)[(2+n_inter_images)*i:(2+n_inter_images)*(i+1)]:
        temp_nrgs.append(entry.get_potential_energy())
        temp_forces.append(entry.get_forces())
        temp_images.append(entry)
    all_nrgs.append(temp_nrgs)
    all_forces.append(temp_forces)
    all_images.append(temp_images)

In [ ]:


fig = plt.figure()
ax = fig.add_subplot(111)
xs = np.arange(2+n_inter_images)
for i in range(60, 66):
    ax.plot(xs, all_nrgs[i], '-o')
ax.set_xlabel('images from initial to final')
ax.set_ylabel('EMT energy (eV)')
ax.legend([f'iter {i}' for i in range(10, 14)])

In [ ]:
traj = Trajectory('./datasets/MDMin-trajs/Pt-MDmin-after-step0.traj', 'r')
nrgs = []
forces = []

for i in range(51):
    temp_nrgs = []
    temp_frs = []
    for image in traj[i*11 : (i+1)*11]:
        image.set_calculator(EMT())
        temp_nrgs.append(image.get_potential_energy())
        temp_frs.append(abs(image.get_forces()[168:, :]).max())
    nrgs.append(temp_nrgs)
    forces.append(temp_frs)



In [ ]:
from ase.calculators.singlepoint import SinglePointCalculator as SPC

new_db = connect('./datasets/Pt-second-images.db')
new_PtAu_db = connect('./datasets/PtAu-second-image.db')

for entry in new_db.select():
    atoms = entry.toatoms()
    energy = atoms.get_potential_energy()
    forces = atoms.get_forces()
    dummy = atoms.copy()
    for j in range(-7, 0):
        dummy[j].symbol = 'Au'
    dummy.set_calculator(SPC(dummy, energy=energy, forces=forces))
    new_PtAu_db.write(dummy)
    

In [ ]:
cov_db = connect('./datasets/Pt-converge-images.db')
cov_nrgs = []
cov_frs = []
for entry in cov_db.select():
    atoms = entry.toatoms()
    atoms.set_calculator(EMT())
    cov_nrgs.append(atoms.get_potential_energy())
    cov_frs.append(abs(atoms.get_forces()[168:, :]).max())

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)


# ax.plot(range(11), forces[0], '-o')

ax.plot(range(11), nrgs[40], '-o')
ax.plot(range(11), nrgs[50], '-o')
ax.plot(range(11), cov_nrgs, '-o')
ax.legend(['40', '50', 'cov'])
